# Data Cleaning and Merge

The notebook details the process of cleaning the mental health and greenspace datasets, merging them, and analyzing key findings. Ultimately, it outputs a regionally and divisionally labeled merged dataframe as `merged_cleaned_data.csv`, located at `data/cleaned/` directory.

## Content
- Cleaning Mental Health Dataset (@ Shuting, remove assignment before submission)
- Cleaning Greenspace Dataset
- Adding Regions and Divisions Labels (@ Shuting)
- Merging Mental Health and Greenspace Datases
- Output to CSV

In [224]:
import file_path as fp
import clean_merge_module as cm
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [225]:
# example of file_path from file_path.py
fp.mh_file

'../data/raw_data/500_Cities__City-level_Data__GIS_Friendly_Format___2017_release_20240514.csv'

## Cleaning Mental Health and Key Findings

We will begin by cleaning the raw mental health dataset, explain the meaning of important columns and present our key findings about this dataset.


In [226]:
# load the file
mh_raw = cm.load_file_df(fp.mh_file)

# remove other chronic diseases columns
mh_data = cm.mh_remove_chronics(mh_raw)
mh_data.head(2)

,StateAbbr,PlaceName,PlaceFIPS,Population2010,MHLTH_CrudePrev,MHLTH_Crude95CI,MHLTH_AdjPrev,MHLTH_Adj95CI,Geolocation
0,AL,Birmingham,107000,212237,15.6,"(15.4, 15.8)",15.6,"(15.4, 15.8)","(33.52756637730, -86.7988174678)"
1,AL,Hoover,135896,81619,10.4,"(10.1, 10.7)",10.4,"(10.1, 10.7)","(33.37676027290, -86.8051937568)"


The dataset has been refined by excluding other chronic diseases, resulting in a dataframe focused on mental health. The remained features are explained in the table below:

|Features|Type|Meaning|
|--|--|--|
|StateAbbr|Plain Text|State abbreviation|
|PlaceName|Plain Text|City name|
|PlaceFIPS|Number|City FIPS Code|
|Population2010|Number|2010 Census population count|
|MHLTH_CrudePrev|Number|Crude prevalence of poor mental health for 14 days or more among adults aged 18 years and older, 2015. <br> Crude prevalence represents the ratio of the total number of responses of 'not good' to the total number of valid responses (excluding those who refused to answer, provided no response, or indicated 'don’t know/not sure').|
|MHLTH_Crude95CI|Plain Text|Estimated 95% confidence interval for crude prevalence|
|MHLTH_AdjPrev|Number|Age-adjusted prevalence, standardized by the direct method to the year 2000 standard U.S. population, distribution 9. `[1]` |
|MHLTH_Adj95CI|Plain Text|Estimated 95% Confidence interval for age-adjusted prevalence|
|Geolocation|Plain Text|Latitude, longitude of city centroid|

Further cleaning and manipulation will be necessary as some features are less useful or stored in an incorrect format:

Removing Features:

- PlaceFIPS: We will use `PlaceName` (city name) as primary key, hence this is less important.
- MHLTH_CrudePrev, MHLTH_Crude95CI: We will use age-adjusted prevalence because it represents standardized prevalence.

Transforming Format:

- Geolocation: Geolocation needs to be converted into a list of two floats representing latitude and longitude.

`[1]` The direct method, aligned with the year 2000 standard U.S. population distribution 9, is a statistical technique used to adjust for age differences by assigning different weights to various age groups. This method is a policy mandated by the Department of Health and Human Services (DHHS) across all its agencies, aiming to enhance the comparability of age-adjusted rates among data systems.[(reference)](https://www.cdc.gov/places/measure-definitions/health-status/index.html#mental-health) Distribution 9 indicates that this age-adjusted prevalence uses the weighting factors provided by Distribution 9. For more information about the weight, check [page 3](https://www.cdc.gov/nchs/data/statnt/statnt20.pdf).

In [227]:
mh_cleaned = cm.mh_clean_transfrom(
    mh_data,
    col_lst=["PlaceFIPS", "MHLTH_CrudePrev", "MHLTH_Crude95CI"],
    trans_col="Geolocation",
)

# save the cleaned file
cm.save_csv(mh_cleaned, fp.mh_cleaned)

mh_cleaned.head(2)

../data/cleaned_data/mh_cleaned.csv already exists.


,StateAbbr,PlaceName,Population2010,MHLTH_AdjPrev,MHLTH_Adj95CI,Geolocation
0,AL,Birmingham,212237,15.6,"(15.4, 15.8)","[33.5275663773, -86.7988174678]"
1,AL,Hoover,81619,10.4,"(10.1, 10.7)","[33.3767602729, -86.8051937568]"


In [228]:
# presenting top 5 states with highest MHLTH_AdjPrev
# the higher the MHLTH_AdjPrev, the worse the mental health condition
cm.show_top5(mh_cleaned, "MHLTH_AdjPrev")

,StateAbbr,PlaceName,Population2010,MHLTH_AdjPrev,MHLTH_Adj95CI,Geolocation
275,MA,New Bedford,95072,18.3,"(18.0, 18.6)","[41.6712667258, -70.9441204537]"
271,MA,Fall River,88857,18.2,"(17.8, 18.5)","[41.7139907598, -71.0996396919]"
279,MA,Springfield,153060,17.5,"(17.3, 17.7)","[42.1154977999, -72.5395254143]"
390,PA,Reading,88082,17.4,"(17.1, 17.6)","[40.3399678686, -75.9266128837]"
285,MI,Flint,102434,17.4,"(17.2, 17.6)","[43.0236339386, -83.6920640313]"


In [229]:
# treemap to present an overview of the mental health prevalence by state and city
mh_treemap = cm.mh_plotly_treemap(mh_cleaned)
mh_treemap.show()

### Key Findings

- **Top 5 Cities with Severe Mental Health Issues**: New Bedford, MA (18.3%), Fall River, MA (18.2%), Springfield, MA (17.5%), Reading, PA (17.4%), and Flint, MI (17.4%).

- **State-Level Analysis**: While Massachusetts (MA) might intuitively seem the most affected state, it is actually the second, with an average mental health issue prevalence of 15.06%. Ohio (OH) ranks highest with an average prevalence of 15.37%.

- **Distribution of Severe Cases**: Massachusetts has a higher concentration of cities with severe mental health challenges; three out of the top 13 cities have prevalences over 17%. In contrast, Ohio has only one city above this threshold.

- **Impact of Sample Variation**: The inclusion of cities like Newton, MA (9.2%), which has a lower prevalence, impacts the average for Massachusetts. This demonstrates how city selection can significantly affect state-level averages and potentially introduce biases if not considered carefully.

- **Potential Bias**: Analyzing data at a larger geographic scale than the city level might introduce selection bias, which is a significant limitation of this study.

## Cleaning Greenspace Dataset

Next we will clean the EU dataset that provides information about Greenspace and other environmental factors in designated Urban Centers.

Start by reading in the raw csv as a dataframe and exploring some information about the data, such as shape and basic stats.

In [230]:
rawdf = pd.read_csv(
    "../data/raw_data/GreenspaceDownload/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.csv",
    encoding="unicode_escape",
    low_memory=False,
)


In [231]:
rawdf.shape

(36902, 160)

In [232]:
rawdf.describe()

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,XBRDR,...,EX_SS_AREA,EX_SS_B75,EX_SS_B90,EX_SS_B00,EX_SS_B15,EX_SS_P75,EX_SS_P90,EX_SS_P00,EX_SS_P15,SDG_A2G14
count,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,...,13135.000000,13135.000000,13135.000000,13135.000000,13135.000000,1.313500e+04,1.313500e+04,1.313500e+04,1.313500e+04,13135.000000
mean,6568.000000,1.176323,50.260830,21.100424,51.795521,21.165498,51.882079,21.132864,51.838877,0.012181,...,5.875828,2.026060,2.289090,2.585256,2.735535,1.428122e+04,1.920504e+04,2.227410e+04,2.603680e+04,0.290790
std,3791.892228,0.429571,189.169066,17.966437,58.682289,17.974479,58.675668,17.970380,58.679035,0.109698,...,61.114603,31.431693,33.614654,36.488360,37.881205,1.810299e+05,2.337269e+05,2.678972e+05,3.149791e+05,0.291329
min,1.000000,0.000000,1.000000,-54.825090,-158.043016,-54.785331,-157.730529,-54.803854,-157.893497,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.012064
25%,3284.500000,1.000000,8.000000,10.760656,19.585115,10.821730,19.615632,10.792640,19.600961,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.054903
50%,6568.000000,1.000000,18.000000,24.368125,73.429889,24.418414,73.499483,24.390754,73.468653,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.182939
75%,9851.500000,1.000000,37.000000,32.654868,90.288306,32.732591,90.383252,32.702407,90.340966,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.461405
max,13135.000000,2.000000,6622.000000,69.303014,178.382096,69.362302,178.526063,69.333682,178.461255,1.000000,...,3487.000000,1987.376221,2141.008545,2326.824463,2419.250488,1.081289e+07,1.227889e+07,1.271757e+07,1.504941e+07,1.003432


In [233]:
rawdf.dtypes

ID_HDC_G0      float64
QA2_1V         float64
AREA           float64
BBX_LATMN      float64
BBX_LONMN      float64
                ...   
EX_EQ19_Q       object
EX_HW_IDX       object
SDG_LUE9015     object
SDG_A2G14      float64
SDG_OS15MX      object
Length: 160, dtype: object

Now that we know a bit more about our dataframe, we can narrow down the columns we want to explore in our EDA. We chose the following columns based on their description in the source data documentation, as these features all seemed like general factors that could affect a person's mental health in a given urban space. We also made a point not to include columns that assigned values at a country level because of our plans to filter the dataset only to the United States for this project.

In [234]:
cols_to_keep = [
    "AREA",
    "GCPNT_LAT",
    "GCPNT_LON",
    "CTR_MN_NM",
    "UC_NM_MN",
    "UC_NM_LST",  # general
    "E_BM_NM_LST",
    "E_SL_LST",
    "EL_AV_ALS",
    "E_WR_P_14",
    "E_WR_T_14",
    "E_GR_AV14",
    "E_GR_AT14",  # geography
    "P15",
    "B15",
    "BUCAP15",
    "NTL_AV",
    "GDP15_SM",  # socio
    "E_EC2E_R15",
    "E_EC2E_I15",
    "E_EC2E_T15",
    "E_EC2E_A15",
    "E_EC2O_R15",
    "E_EC2O_I15",  # emissions1
    "E_EC2O_A15",
    "E_EPM2_R15",
    "E_EPM2_I15",
    "E_EPM2_T15",
    "E_EPM2_A15",
    "E_CPM2_T14",  # emissions2
    "SDG_A2G14",
    "SDG_OS15MX",
    "SDG_LUE9015",
    "EX_HW_IDX",  # landuse
]

# create new dataframe with only the selected columns
gs_df = rawdf[cols_to_keep]

# filter the data to only include Urban Centers in the United States, as our mental health data is limited to US Cities.
gs_df = gs_df[gs_df["CTR_MN_NM"] == "United States"]

# replace confusing and messy values with NaN
gs_df.replace(
    to_replace=["?", "??", "???", "NAN"],
    value=[np.nan, np.nan, np.nan, np.nan],
    inplace=True,
)

# assign more readable column names to make analysis easier
gs_df.rename(
    columns={
        "AREA": "Urban Center Area",
        "GCPNT_LAT": "Latitude",
        "GCPNT_LON": "Longitude",
        "CTR_MN_NM": "Country",
        "UC_NM_MN": "Urban Center",
        "UC_NM_LST": "Cities in Urban Center",
        "E_BM_NM_LST": "Biome",
        "E_SL_LST": "Soil Group",
        "EL_AV_ALS": "Avg Elevation",
        "E_WR_P_14": "Avg Precipitation",
        "E_WR_T_14": "Avg Temp",
        "E_GR_AV14": "Avg Greenness",
        "E_GR_AT14": "Total Green Area",
        "P15": "Population",
        "B15": "Total Built-up Area",
        "BUCAP15": "Built-up Area per capita",
        "NTL_AV": "Avg Nighttime Light Emission",
        "GDP15_SM": "Sum of GDP",
        "E_EC2E_R15": "TCNSCE Residential",  # TCNSCE stands for "total co2 non short cycle emissions"
        "E_EC2E_I15": "TCNSCE Industry",
        "E_EC2E_T15": "TCNSCE Transport",
        "E_EC2E_A15": "TCNSCE Agriculture",
        "E_EC2O_R15": "TCSCOE Residential",  # TCSCOE stands for "total co2 short cycle organic emissions"
        "E_EC2O_I15": "TCSCOE Industry",
        "E_EC2O_A15": "TCSCOE Agriculture",
        "E_EPM2_R15": "Particulate Matter Emissions Residential",
        "E_EPM2_I15": "Particulate Matter Emissions Industry",
        "E_EPM2_T15": "Particulate Matter Emissions Transport",
        "E_EPM2_A15": "Particulate Matter Emissions Agriculture",
        "E_CPM2_T14": "Total Concertation of Particulate Matter",
        "SDG_A2G14": "% of Pop in High Green Area",
        "SDG_OS15MX": "% of Open Spaces",
        "SDG_LUE9015": "Land Use Efficiency",
        "EX_HW_IDX": "Max Magnitude of Heatwaves",
    },
    inplace=True,
)

# let's explore our updated dataframe
gs_df.head()


,Urban Center Area,Latitude,Longitude,Country,Urban Center,Cities in Urban Center,Biome,Soil Group,Avg Elevation,Avg Precipitation,...,TCSCOE Agriculture,Particulate Matter Emissions Residential,Particulate Matter Emissions Industry,Particulate Matter Emissions Transport,Particulate Matter Emissions Agriculture,Total Concertation of Particulate Matter,% of Pop in High Green Area,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves
0,185.0,21.340678,-157.893497,United States,Honolulu,Honolulu; Waipahu; Pearl City; Aiea,Tropical and Subtropical Dry Broadleaf Forests,Vertisols,52.29643514,741.6250153,...,3.078803527,40.14455643,174.221274,23.53676553,0.047533547,5.765125,0.226415,56.41,0.074385203,NaN
2,55.0,34.923123,-120.434372,United States,Santa Maria,Santa Maria,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,87.12809223,245.1750011,...,8.266219641,8.503971075,35.140393,3.014212862,0.047214542,11.630900,0.040129,23.64,0.481144026,2.791739941
3,48.0,36.607720,-121.882378,United States,Monterey,Monterey,"Mediterranean Forests, Woodlands, and Scrub",Phaeozems,38.54796203,324.9250069,...,1.07850128,7.497389813,11.707452,1.972619542,0.008828798,10.932750,0.138683,42.17,0.444839872,NaN
4,60.0,34.427664,-119.743693,United States,Santa Barbara,Santa Barbara,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,38.29809451,183.9250069,...,1.329586573,14.04887269,30.528288,3.263458168,0.013192485,13.803700,0.061348,36.5,0.55676245,4.255020142
5,57.0,36.971274,-121.978684,United States,Santa Cruz,Santa Cruz,"Mediterranean Forests, Woodlands, and Scrub",Phaeozems,24.08581144,324.9250069,...,0.071545386,11.32560598,24.377837,3.99150779,0.001443604,11.868250,0.109287,39.46,0.317594332,1.51970005


Let's check the dataframe to make sure there aren't any strange characters that could cause a mismatch between our datasets when we join them on city name. I amended this cell to check for the following characters: "?", "[", ",", ":", ";" but only "?" and "[" returned any data.

In [235]:
checker = gs_df[gs_df["Urban Center"].str.contains("?", regex=False)]
checker


,Urban Center Area,Latitude,Longitude,Country,Urban Center,Cities in Urban Center,Biome,Soil Group,Avg Elevation,Avg Precipitation,...,TCSCOE Agriculture,Particulate Matter Emissions Residential,Particulate Matter Emissions Industry,Particulate Matter Emissions Transport,Particulate Matter Emissions Agriculture,Total Concertation of Particulate Matter,% of Pop in High Green Area,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves
482,168.0,38.777312,-90.611861,United States,O?Fallon,O?Fallon; Saint Charles; Saint Peters; Cottlev...,"Temperate Grasslands, Savannas, and Shrublands",Phaeozems,163.0952796,1057.825027,...,1832.278462,114.1057715,281.217959,45.87761046,7.098453931,11.3387,0.790093,75.6,1.532844781,17.70470047


In [236]:
# replace the "?" with an apostrophe, as the city is commonly known as "O'Fallon"
a1 = gs_df.loc[482]["Cities in Urban Center"]
a1replace = a1.replace("?", "'")


In [237]:
# also replace "Minneapolis [Saint Paul]" with "Minneapolis so that it can be matched to the mental health dataset"
gs_df.at[482, "Urban Center"] = "O'Fallon"
gs_df.at[482, "Cities in Urban Center"] = a1replace
gs_df.at[553, "Urban Center"] = "Minneapolis"


Now let's import the mental health dataset so that we can see what cities have data in both datasets. In the next few steps we will explode out the Greenspace dataset so that there is one row per city in an urban center. This will allow us to later merge the mental health data with the greenspace data by city, then reaggregate back into urban centers.

In [238]:
mhdf = pd.read_csv(
    "../data/raw_data/500_Cities__City-level_Data__GIS_Friendly_Format___2017_release_20240514.csv"
)
mhdf.columns


Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'Population2010',
       'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI', 'ACCESS2_AdjPrev',
       'ACCESS2_Adj95CI', 'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI',
       ...
       'SLEEP_Adj95CI', 'STROKE_CrudePrev', 'STROKE_Crude95CI',
       'STROKE_AdjPrev', 'STROKE_Adj95CI', 'TEETHLOST_CrudePrev',
       'TEETHLOST_Crude95CI', 'TEETHLOST_AdjPrev', 'TEETHLOST_Adj95CI',
       'Geolocation'],
      dtype='object', length=117)

In [239]:
# get a list of unique city names in the mental health dataset to filter down the greenspace dataset
mh_cities = (mhdf["PlaceName"].unique()).tolist()
mh_cities


['Birmingham',
 'Hoover',
 'Huntsville',
 'Mobile',
 'Montgomery',
 'Tuscaloosa',
 'Anchorage',
 'Avondale',
 'Chandler',
 'Gilbert',
 'Glendale',
 'Mesa',
 'Peoria',
 'Phoenix',
 'Scottsdale',
 'Surprise',
 'Tempe',
 'Tucson',
 'Yuma',
 'Fayetteville',
 'Fort Smith',
 'Jonesboro',
 'Little Rock',
 'Springdale',
 'Alameda',
 'Alhambra',
 'Anaheim',
 'Antioch',
 'Apple Valley',
 'Bakersfield',
 'Baldwin Park',
 'Bellflower',
 'Berkeley',
 'Buena Park',
 'Burbank',
 'Carlsbad',
 'Carson',
 'Chico',
 'Chino',
 'Chino Hills',
 'Chula Vista',
 'Citrus Heights',
 'Clovis',
 'Compton',
 'Concord',
 'Corona',
 'Costa Mesa',
 'Daly City',
 'Downey',
 'El Cajon',
 'Elk Grove',
 'El Monte',
 'Escondido',
 'Fairfield',
 'Folsom',
 'Fontana',
 'Fremont',
 'Fresno',
 'Fullerton',
 'Garden Grove',
 'Hawthorne',
 'Hayward',
 'Hemet',
 'Hesperia',
 'Huntington Beach',
 'Indio',
 'Inglewood',
 'Irvine',
 'Lake Forest',
 'Lakewood',
 'Lancaster',
 'Livermore',
 'Long Beach',
 'Los Angeles',
 'Lynwood',
 

In [240]:
# get number of rows in dataset before filtering
print(gs_df.shape)

# make a column to copy the cities in the urban center so that we can retain this information for later
gs_df["Cities in Urban Center_copy"] = gs_df["Cities in Urban Center"]

# split the "Cities in Urban Center" column into a list of strings
gs_df["Cities in Urban Center"] = gs_df["Cities in Urban Center"].str.split(";")

# explode the dataset into one row for each city in the urban center and reset the index. Retain the original index for later grouping.
gs_df = gs_df.explode("Cities in Urban Center")
gs_df.reset_index(inplace=True, drop=False)
gs_df.rename(columns={"index": "UC Grouping"}, inplace=True)

# strip leading and trailing whitespace from the city column
gs_df["PlaceName"] = gs_df["Cities in Urban Center"].str.strip()
gs_df = gs_df.drop("Cities in Urban Center", axis=1)


(324, 34)


In [241]:
# get a list of the rows whose cities are present in the mental health dataset, and remove other rows.
ucgroup = gs_df[gs_df["PlaceName"].isin(mh_cities)]
ucgrouplist = ucgroup.index.tolist()
gs_df = gs_df[gs_df.index.isin(ucgrouplist)]
gs_df.head()


,UC Grouping,Urban Center Area,Latitude,Longitude,Country,Urban Center,Biome,Soil Group,Avg Elevation,Avg Precipitation,...,Particulate Matter Emissions Industry,Particulate Matter Emissions Transport,Particulate Matter Emissions Agriculture,Total Concertation of Particulate Matter,% of Pop in High Green Area,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves,Cities in Urban Center_copy,PlaceName
0,0,185.0,21.340678,-157.893497,United States,Honolulu,Tropical and Subtropical Dry Broadleaf Forests,Vertisols,52.29643514,741.6250153,...,174.221274,23.53676553,0.047533547,5.765125,0.226415,56.41,0.074385203,NaN,Honolulu; Waipahu; Pearl City; Aiea,Honolulu
4,2,55.0,34.923123,-120.434372,United States,Santa Maria,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,87.12809223,245.1750011,...,35.140393,3.014212862,0.047214542,11.630900,0.040129,23.64,0.481144026,2.791739941,Santa Maria,Santa Maria
6,4,60.0,34.427664,-119.743693,United States,Santa Barbara,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,38.29809451,183.9250069,...,30.528288,3.263458168,0.013192485,13.803700,0.061348,36.5,0.55676245,4.255020142,Santa Barbara,Santa Barbara
8,6,54.0,36.688991,-121.640831,United States,Salinas,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,21.30891932,324.9250069,...,65.973825,6.895040733,0.059791583,12.376300,0.076114,24.61,0.843804111,11.39150047,Salinas,Salinas
9,7,136.0,34.217486,-119.209132,United States,Oxnard,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,21.89917854,238.6750031,...,118.460613,16.89482998,0.072824232,17.010900,0.036199,28.65,0.43558362,NaN,Oxnard; Ventura,Oxnard


Unfortunately there are cases where there are two cities with the same name in different states, for example Portland Oregon and Portland Washington. In order to merge the datasets correctly, we will need to join on both city name and state abbreviation. We use the state boundary polygon data provided with the mental health dataset to detect which state each urban center belongs to based on its latitude and longitude columns.

In [242]:
# import state boundaries file and investigate
stateboundaries = gpd.read_file(
    "../data/raw_data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
)
stateboundaries.head()


,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [243]:
# create a function that finds an urban center's state from its latitude and longitude columns so that we can apply to the whole dataframe
def statefinder(row):
    point = Point(row["Longitude"], row["Latitude"])
    state = stateboundaries[stateboundaries.contains(point)]

    if not state.empty:
        return state.iloc[0]["STUSPS"]  # return state abbreviation
    else:
        return np.nan  # otherwise return NaN


In [244]:
# apply this function to the dataframe to create the State column
gs_df["State"] = gs_df.apply(statefinder, axis=1)


## Adding Region and Division Labels

To finish up the individual dataset cleaning, we will add indicators of Region and Division corresponding to the urban center's assigned state. This will help later with exploratory analysis and grouping findings across the US.

In [245]:
def us_division():
    """
    Returns a dictionary of US divisions and their respective states.
    """
    us_divisions = {
        "New England": ["CT", "ME", "MA", "NH", "RI", "VT"],
        "Middle Atlantic": ["NJ", "NY", "PA"],
        "East North Central": ["IL", "IN", "MI", "OH", "WI"],
        "West North Central": ["IA", "KS", "MN", "MO", "NE", "ND", "SD"],
        "South Atlantic": ["DE", "FL", "GA", "MD", "NC", "SC", "VA", "WV", "DC"],
        "East South Central": ["AL", "KY", "MS", "TN"],
        "West South Central": ["AR", "LA", "OK", "TX"],
        "Mountain": ["AZ", "CO", "ID", "MT", "NV", "NM", "UT", "WY"],
        "Pacific": ["AK", "CA", "HI", "OR", "WA"],
    }
    return us_divisions


In [246]:
def us_region():
    """
    Returns a dictionary of US regions and their respective states.
    """

    us_regions = {
        "West": [
            "AK",
            "AZ",
            "CA",
            "CO",
            "HI",
            "ID",
            "MT",
            "NV",
            "NM",
            "OR",
            "UT",
            "WA",
            "WY",
        ],
        "Midwest": [
            "IL",
            "IN",
            "IA",
            "KS",
            "MI",
            "MN",
            "MO",
            "NE",
            "ND",
            "OH",
            "SD",
            "WI",
        ],
        "Northeast": ["CT", "DE", "ME", "MD", "MA", "NH", "NJ", "NY", "PA", "RI", "VT"],
        "South": [
            "AL",
            "AR",
            "FL",
            "GA",
            "KY",
            "LA",
            "MS",
            "NC",
            "OK",
            "SC",
            "TN",
            "TX",
            "VA",
            "WV",
            "DC",
        ],
    }
    return us_regions


In [247]:
def apply_geo_labels(df, label_col_name, label_dict, base_col):
    """
    Apply labels based on existing column.
    Input df, name for labeled column, label dictionary, and based column.
    Returns the dataframe with the labeled column.
    """
    new_df = df.copy()
    new_df[label_col_name] = ["None" for x in range(len(df))]
    for key, value in label_dict.items():
        new_df.loc[new_df[base_col].isin(value), label_col_name] = key
    return new_df


In [248]:
# apply the above functions to create Region and Division labels for each dataframe row

region_dic = us_region()
div_dic = us_division()

gs_df = apply_geo_labels(gs_df, "Region", region_dic, "State")
gs_df = apply_geo_labels(gs_df, "Division", div_dic, "State")


In [249]:
# let's do a final check to ensure that we don't have any blanks in our added columns before merging the datasets
gs_df[gs_df["State"].isna()].head()


,UC Grouping,Urban Center Area,Latitude,Longitude,Country,Urban Center,Biome,Soil Group,Avg Elevation,Avg Precipitation,...,Total Concertation of Particulate Matter,% of Pop in High Green Area,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves,Cities in Urban Center_copy,PlaceName,State,Region,Division


In [250]:
# exporting to CSV for easy access and reference outside this notebook
gs_df.to_csv("../data/cleaned_data/greenspace_cleaned.csv")


## Merge Datasets

Now we merge our cleaned mental health dataset (mh_cleaned) and our cleaned greenspace dataset (gs_df) by performing an inner join on the City and State columns.

In [251]:
# rename a few columns for clarity and to ensure that the datasets can be joined correctly
mh_cleaned.rename(
    columns={
        "StateAbbr": "State",
        "Population2010": "MH_Population",
        "MHLTH_AdjPrev": "MH_Score",
    },
    inplace=True,
)
mh_cleaned.head()


,State,PlaceName,MH_Population,MH_Score,MHLTH_Adj95CI,Geolocation
0,AL,Birmingham,212237,15.6,"(15.4, 15.8)","[33.5275663773, -86.7988174678]"
1,AL,Hoover,81619,10.4,"(10.1, 10.7)","[33.3767602729, -86.8051937568]"
2,AL,Huntsville,180105,13.4,"(13.2, 13.7)","[34.6989692671, -86.6387042882]"
3,AL,Mobile,195111,15.0,"(14.9, 15.2)","[30.6776248648, -88.1184482714]"
4,AL,Montgomery,205764,14.8,"(14.6, 15.1)","[32.3472645333, -86.2677059552]"


In [252]:
# remove additional unnecessary columns
gs_df.drop(columns=["Country"], inplace=True)
mh_cleaned.drop(columns=["Geolocation", "MHLTH_Adj95CI"], inplace=True)


Now we can merge the two datasets by performing an inner join on the State and PlaceName (city)

In [253]:
df = pd.merge(mh_cleaned, gs_df, on=["PlaceName", "State"], how="inner")
df.head()


,State,PlaceName,MH_Population,MH_Score,UC Grouping,Urban Center Area,Latitude,Longitude,Urban Center,Biome,...,Particulate Matter Emissions Transport,Particulate Matter Emissions Agriculture,Total Concertation of Particulate Matter,% of Pop in High Green Area,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves,Cities in Urban Center_copy,Region,Division
0,AL,Birmingham,212237,15.6,485,221.0,33.509025,-86.823651,Birmingham,Temperate Broadleaf and Mixed Forests,...,28.64959924,0.007313893,10.32690,0.773812,74.85,0.661872059,20.21800041,Birmingham;,South,East South Central
1,AL,Huntsville,180105,13.4,501,89.0,34.726065,-86.609995,Huntsville,Temperate Broadleaf and Mixed Forests,...,6.546264636,0.472177503,10.86525,0.802599,66.37,0.279664898,26.78790092,Huntsville,South,East South Central
2,AL,Mobile,195111,15.0,422,123.0,30.692377,-88.093685,Mobile,Temperate Coniferous Forests,...,12.98888705,0.018721169,10.41785,0.822213,63.32,0.750273369,9.628219604,Mobile,South,East South Central
3,AL,Montgomery,205764,14.8,481,154.0,32.360134,-86.249187,Montgomery,Temperate Broadleaf and Mixed Forests,...,10.83643126,0.49016586,11.33750,0.747151,75.47,1.659945361,14.78829956,Montgomery,South,East South Central
4,AL,Tuscaloosa,90468,15.5,470,45.0,33.199564,-87.553567,Tuscaloosa,Temperate Broadleaf and Mixed Forests,...,5.127922857,0.163041035,11.48650,0.411681,41.24,0.783592015,14.61019993,Tuscaloosa,South,East South Central


We aren't quite done yet - since the greenspace data was provided to us at the Urban Center level (in some cases indicating a group of cities), we don't want to split out that data into individual cities as we don't know the distribution of the features among cities within an Urban Center. We decided to roll up the mental health data to the Urban Center level in order for our analysis of the combined dataset to have a consistent basis. We will re-aggregate the dataset to the Urban Center level in the following steps.

In [254]:
# we can't aggregate the dataset with nonnumeric data, so we pull out all columns with object type data and put them into their own temporary dataset.
df_alpha = df[
    [
        "UC Grouping",
        "State",
        "Urban Center",
        "Biome",
        "Soil Group",
        "Cities in Urban Center_copy",
        "Region",
        "Division",
    ]
].copy()
df.drop(
    columns=[
        "State",
        "Urban Center",
        "PlaceName",
        "Biome",
        "Soil Group",
        "Cities in Urban Center_copy",
        "Region",
        "Division",
    ],
    inplace=True,
)


Now we get our list of columns to aggregate as mean - since we exploded the greenspace dataset earlier on the PlaceName column, taking the mean of all the greenspace columns will give us back the original values for each Urban Center. The real work here is in the aggregation of the mental health columns; the population will be summed since we are grouping cities into urban centers, and the mental health scores will be averaged to provide us with an average MH score for an urban center.

In [255]:
# convert remaining object columns with numeric values to numeric types
temp = df.select_dtypes(include="object")
templist = list(temp.columns)
df[templist] = df[templist].apply(pd.to_numeric)


In [256]:
cols = list(df.columns)
cols.remove("MH_Population")
print(cols)


['MH_Score', 'UC Grouping', 'Urban Center Area', 'Latitude', 'Longitude', 'Avg Elevation', 'Avg Precipitation', 'Avg Temp', 'Avg Greenness', 'Total Green Area', 'Population', 'Total Built-up Area', 'Built-up Area per capita', 'Avg Nighttime Light Emission', 'Sum of GDP', 'TCNSCE Residential', 'TCNSCE Industry', 'TCNSCE Transport', 'TCNSCE Agriculture', 'TCSCOE Residential', 'TCSCOE Industry', 'TCSCOE Agriculture', 'Particulate Matter Emissions Residential', 'Particulate Matter Emissions Industry', 'Particulate Matter Emissions Transport', 'Particulate Matter Emissions Agriculture', 'Total Concertation of Particulate Matter', '% of Pop in High Green Area', '% of Open Spaces', 'Land Use Efficiency', 'Max Magnitude of Heatwaves']


In [257]:
# group the dataframe by UC Grouping, which came from the exploded index in the greenspace dataset;
# rows that originated from the same row will have the same UC Grouping number.
aggregations = {"MH_Population": "sum"}
for i in cols:
    aggregations[i] = "mean"
grouped = df.groupby(by="UC Grouping").agg(aggregations)


In [258]:
# apply some cleanup
grouped["UC Grouping"] = grouped["UC Grouping"].astype(int)
grouped.reset_index(drop=True, inplace=True)
grouped.head()


,MH_Population,MH_Score,UC Grouping,Urban Center Area,Latitude,Longitude,Avg Elevation,Avg Precipitation,Avg Temp,Avg Greenness,...,TCSCOE Agriculture,Particulate Matter Emissions Residential,Particulate Matter Emissions Industry,Particulate Matter Emissions Transport,Particulate Matter Emissions Agriculture,Total Concertation of Particulate Matter,% of Pop in High Green Area,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves
0,953207,9.2,0,185.0,21.340678,-157.893497,52.296435,741.625015,23.526622,0.369290,...,3.078804,40.144556,174.221274,23.536766,0.047534,5.765125,0.226415,56.41,0.074385,NaN
1,99553,15.0,2,55.0,34.923123,-120.434372,87.128092,245.175001,14.718191,0.312846,...,8.266220,8.503971,35.140393,3.014213,0.047215,11.630900,0.040129,23.64,0.481144,2.79174
2,88410,11.1,4,60.0,34.427664,-119.743693,38.298095,183.925007,15.376907,0.362785,...,1.329587,14.048873,30.528288,3.263458,0.013192,13.803700,0.061348,36.50,0.556762,4.25502
3,150441,14.7,6,54.0,36.688991,-121.640831,21.308919,324.925007,15.274110,0.339631,...,8.590625,16.569708,65.973825,6.895041,0.059792,12.376300,0.076114,24.61,0.843804,11.39150
4,197899,13.6,7,136.0,34.217486,-119.209132,21.899179,238.675003,17.053577,0.299903,...,15.607472,30.233594,118.460613,16.894830,0.072824,17.010900,0.036199,28.65,0.435584,NaN


Now we need to add back in the object based columns to the re-aggregated dataset. Luckily these columns all come from the greenspace dataset, meaning that when the dataframe was exploded these columns were duplicated. To rejoin the object based columns, we just need to remove duplicates and perform an inner join on UC Grouping.

In [259]:
# look at shapes before and after dropping duplicates
print(df_alpha.shape)
df_alpha.drop_duplicates(keep="first", inplace=True)
print(df_alpha.shape)


(329, 8)
(228, 8)


In [260]:
# join the object based columns back on to the merged, cleaned, Urban Center-level dataframe
dfagg = pd.merge(grouped, df_alpha, on=["UC Grouping"], how="inner")
print(dfagg.shape)
dfagg.head()


(228, 39)


,MH_Population,MH_Score,UC Grouping,Urban Center Area,Latitude,Longitude,Avg Elevation,Avg Precipitation,Avg Temp,Avg Greenness,...,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves,State,Urban Center,Biome,Soil Group,Cities in Urban Center_copy,Region,Division
0,953207,9.2,0,185.0,21.340678,-157.893497,52.296435,741.625015,23.526622,0.369290,...,56.41,0.074385,NaN,HI,Honolulu,Tropical and Subtropical Dry Broadleaf Forests,Vertisols,Honolulu; Waipahu; Pearl City; Aiea,West,Pacific
1,99553,15.0,2,55.0,34.923123,-120.434372,87.128092,245.175001,14.718191,0.312846,...,23.64,0.481144,2.79174,CA,Santa Maria,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,Santa Maria,West,Pacific
2,88410,11.1,4,60.0,34.427664,-119.743693,38.298095,183.925007,15.376907,0.362785,...,36.50,0.556762,4.25502,CA,Santa Barbara,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,Santa Barbara,West,Pacific
3,150441,14.7,6,54.0,36.688991,-121.640831,21.308919,324.925007,15.274110,0.339631,...,24.61,0.843804,11.39150,CA,Salinas,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,Salinas,West,Pacific
4,197899,13.6,7,136.0,34.217486,-119.209132,21.899179,238.675003,17.053577,0.299903,...,28.65,0.435584,NaN,CA,Oxnard,"Mediterranean Forests, Woodlands, and Scrub",Luvisols,Oxnard; Ventura,West,Pacific


In [261]:
# double check that there are no more duplicates or cities that failed to get reaggregated back to the Urban Center level
dfagg[dfagg["UC Grouping"].duplicated()]


,MH_Population,MH_Score,UC Grouping,Urban Center Area,Latitude,Longitude,Avg Elevation,Avg Precipitation,Avg Temp,Avg Greenness,...,% of Open Spaces,Land Use Efficiency,Max Magnitude of Heatwaves,State,Urban Center,Biome,Soil Group,Cities in Urban Center_copy,Region,Division


## Output to CSV

We have output our intermediate cleaned data for the individual datasets as .csv files for later reference and easy use outside this notebook. Our final step is to output our cleaned, merged dataset that we will be performing analysis on in the next notebook. We output this file to our cleaned_data subfolder in the data folder.

In [272]:
dfagg.rename(columns={"UC Grouping": "UC_Grouping"}, inplace=True)
dfagg.to_csv("../data/cleaned_data/merged_cleaned_data.csv")
